### 12개월 데이터로 예측해보기

In [8]:
# 모델 불러오기
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        # input size = 12
        self.rnn = nn.RNN(input_size=12, hidden_size=30, num_layers=2, batch_first=True)
        self.fc = nn.Linear(30, 1)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x[:, -1])
        return x

In [9]:
# Load model
model = RNN()
model.load_state_dict(torch.load('noo_model_12m.pth'))

<All keys matched successfully>

In [10]:
# 모델 사용하기
# 1. 데이터 로드
import pandas as pd
import numpy as np

data = pd.read_csv('../../DATA/SMP_201004_202403_norm.csv')
data.head()
# norm -> value column
data['value'] = data['norm']
data.drop(['날짜', '통합'], axis=1, inplace=True)
data.head()

,norm,denorm,date,value
0,-1.165759,50.705398,2001-04-30,-1.165759
1,-1.123687,52.552364,2001-05-31,-1.123687
2,-1.153413,51.247385,2001-06-30,-1.153413
3,-1.240408,47.428339,2001-07-31,-1.240408
4,-1.357382,42.293200,2001-08-31,-1.357382


In [11]:
def denorms(x):
    return x * 43.89968 + 101.88185

In [19]:
def predict_df(start_date, pred_length):
    # date가 '2023-04-30' 이후이면 에러 출력
    if start_date > '2023-04-30':
        print('date error: 2023-04-30 이전으로만 입력해주세요 (학습 기간 : 12개월)')
        return
    else:
        start_idx = data[data['date'] == start_date].index.item()
        pred_total = data['norm'][start_idx: start_idx+12].tolist()
        for i in range(pred_length):
            values = torch.tensor(pred_total[-12:]).unsqueeze(0).float()
            pred = model(values.unsqueeze(0))
            pred_total = np.append(pred_total, pred.item())
        pred_total_denorm = denorms(pred_total).tolist()
        pred_df = pd.DataFrame({'date': pd.date_range(start_date, periods=pred_length, freq='M'), 'value': pred_total_denorm[-pred_length:]})
        return pred_df
    
predict_df('2023-04-30', 15)

,date,value
0,2023-04-30,142.589826
1,2023-05-31,133.898377
2,2023-06-30,143.171706
3,2023-07-31,133.011273
4,2023-08-31,115.205927
5,2023-09-30,131.433807
6,2023-10-31,127.234014
7,2023-11-30,126.235513
8,2023-12-31,144.897569
9,2024-01-31,133.914435


In [14]:
# 입력 : 시작일, 기간
# 출력 : 데이터프레임 (date, pred)

def predict(start_date, period):
    # Load data
    data = pd.read_csv('../../DATA/SMP_201004_202403_norm.csv')
    
    # 데이터 전처리
    data['date'] = pd.to_datetime(data['date'])
    data.set_index('date', inplace=True)
    data = data[start_date:]
    data = data['value']
    data = data.values

    # 데이터셋 생성
    sequence = 12
    x = []
    y = []
    for i in range(len(data) - sequence):
        x.append(data[i:i+sequence])
        y.append(data[i+sequence])

    x = np.array(x)
    y = np.array(y)

    x = x.reshape(-1, 12, 1)
    y = y.reshape(-1, 1)

    # 예측
    pred = []
    for i in range(period):
        x_input = x[-1]
        x_input = x_input.reshape(1, 12, 1)
        x_input = torch.Tensor(x_input)
        y_pred = model(x_input)
        pred.append(y_pred.item())
        x = np.append(x, y_pred.item()).reshape(-1, 12, 1)

    # 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(y, label='actual')
    plt.plot(pred, label='prediction')
    plt.legend()
    plt.show()

    # 결과
    result = pd.DataFrame()
    result['date'] = pd.date_range(start=start_date, periods=period, freq='M')
    result['pred'] = pred
    result['pred'] = denorms(result['pred'])
    return result

In [15]:
# 예측
result = predict('2019-01-01', 12)
print(result)

KeyError: 'value'